In [8]:
import click
import pystow
import rdflib
import sssom
from curies.dataframe import filter_df_by_prefixes
from tabulate import tabulate
from biomappings.resources import POSITIVES_SSSOM_PATH
import bioregistry
import pyobo
from SPARQLWrapper import SPARQLWrapper, JSON
from rdflib import SKOS

In [2]:
path = pystow.join("nfdi", name="chem-culture.sssom.ttl")
if not path.is_file() and False:
    msdf = sssom.parse_sssom_table(POSITIVES_SSSOM_PATH, prefix_map=bioregistry.get_default_converter())
    # only keep iconclass mappings
    msdf.df = filter_df_by_prefixes(msdf.df, column="subject_id", prefixes=["iconclass"])
    sssom.write_rdf(msdf, path, hydrate=True)

graph = rdflib.Graph()
graph.parse(path)

print(graph)

[a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'Memory']].


In [11]:
for iconclass_uri, object_uri in graph.subject_objects(SKOS.relatedMatch):
    iconclass_reference = bioregistry.parse_iri(iconclass_uri, strict=True)
    iconclass_name = pyobo.get_name(iconclass_reference)
    object_reference = bioregistry.parse_iri(object_uri, strict=True)
    object_name = pyobo.get_name(object_reference)
    print(iconclass_reference.identifier, iconclass_name, object_reference.curie, object_name)


24C31 the sun representing gold chebi:29287 gold atom
11H%28ANTONY%20ABBOT%29411 St. Antony Abbot finds a nugget of gold in the desert chebi:29287 gold atom
22C4%28GOLD%29 colours, pigments, and paints: gold chebi:29287 gold atom
25D13%28GOLD%29 minerals and metals: gold chebi:29287 gold atom
46B332 weighing gold or money chebi:29287 gold atom
49E3911 alchemist trying to make gold chebi:29287 gold atom
71I423 Hiram sends gold to Solomon chebi:29287 gold atom
94E231 gold is found in Palamedes' tent chebi:29287 gold atom
94G42 Hector's body is weighed in gold (Aeschylus) chebi:29287 gold atom
24C32 the moon representing silver chebi:30512 silver atom
25D13%28SILVER%29 minerals and metals: silver chebi:30512 silver atom
71C13211 Abraham weighs out four hundred pieces of silver chebi:30512 silver atom
71D1252 Joseph is sold for twenty pieces of silver chebi:30512 silver atom
71D191 Joseph has his silver cup placed in Benjamin's sack chebi:30512 silver atom
24C33 Mercury representing the me

Demonstrate doing a tiny query over the NFDI4Culture Graph

In [4]:
sparql_wrapper = SPARQLWrapper("https://nfdi4culture.de/sparql")
sparql_wrapper.setReturnFormat(JSON)
sparql_wrapper.setQuery("""
    PREFIX cto: <https://nfdi4culture.de/ontology#>
    SELECT *
    WHERE {
      ?resource cto:subjectConcept ?iconclass .
      OPTIONAL { ?resource rdfs:label ?resourceLabel . }
    }
    LIMIT 5
    """
)
print(sparql_wrapper.queryAndConvert())

{'head': {'link': [], 'vars': ['resource', 'iconclass', 'resourceLabel']}, 'results': {'distinct': False, 'ordered': True, 'bindings': [{'resource': {'type': 'uri', 'value': 'http://www.bildindex.de/document/obj00000008'}, 'iconclass': {'type': 'uri', 'value': 'https://iconclass.org/25F62%28Herring%29'}, 'resourceLabel': {'type': 'literal', 'xml:lang': 'de', 'value': 'Lob des Herings'}}, {'resource': {'type': 'uri', 'value': 'http://www.bildindex.de/document/obj00000008'}, 'iconclass': {'type': 'uri', 'value': 'https://iconclass.org/41C6'}, 'resourceLabel': {'type': 'literal', 'xml:lang': 'de', 'value': 'Lob des Herings'}}, {'resource': {'type': 'uri', 'value': 'http://www.bildindex.de/document/obj00000008'}, 'iconclass': {'type': 'uri', 'value': 'https://iconclass.org/49MM32'}, 'resourceLabel': {'type': 'literal', 'xml:lang': 'de', 'value': 'Lob des Herings'}}, {'resource': {'type': 'uri', 'value': 'http://www.bildindex.de/document/obj00000008'}, 'iconclass': {'type': 'uri', 'value': 

In [5]:
sparql_wrapper = SPARQLWrapper("http://127.0.0.1:8000/")
sparql_wrapper.setReturnFormat(JSON)
sparql_wrapper.setQuery("""
    PREFIX cto: <https://nfdi4culture.de/ontology#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

    SELECT ?iconclass ?resource ?resourceLabel ?target
    WHERE {
        ?iconclass skos:relatedMatch ?target .
        
        SERVICE <https://nfdi4culture.de/sparql> {
            ?resource cto:subjectConcept ?iconclass .
            OPTIONAL { ?resource rdfs:label ?resourceLabel . }
        }
    }
    LIMIT 5
    """
)
print(sparql_wrapper.queryAndConvert())

URLError: <urlopen error [Errno 61] Connection refused>

In [ ]:




""" SERVICE <https://search.nfdi4chem.de/sparql> {} """
sparql = """\
PREFIX cto: <https://nfdi4culture.de/ontology#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>


SELECT *
WHERE {
    ?resource cto:subjectConcept ?iconclass .
    OPTIONAL { ?resource rdfs:label ?resourceLabel . }

    ?iconclass skos:relatedMatch ?target .

    SERVICE <https://nfdi4culture.de/sparql> {
      
    }
}
LIMIT 5
"""
import urllib.error
try:
    records = graph.query(sparql)
    click.echo(tabulate(records))
except urllib.error.HTTPError as e:
    print(type(e))
    print(e.url)
    print(e.code)
    print(e.headers)
    print(dir(e.fp)    )